## 1-Neighbours manuale

In [ ]:
import pandas as pd

In [ ]:
iris = pd.read_csv(filepath_or_buffer = r"C:\Users\ianto\Desktop\Corso Python\Lezione 5\iris.csv",
            sep = ",",
            header = 1)

In [ ]:
nuovo_iris = [5.9,3.0,5.1,1.8]

In [ ]:
iris.head(5)

In [ ]:
iris["distanza"] = ((iris["sepal_length"]-nuovo_iris[0])**2 +
(iris["sepal_width"]-nuovo_iris[1])**2 + 
(iris["petal_length"]-nuovo_iris[2])**2 +
(iris["petal_width"]-nuovo_iris[3])**2)**0.5

In [ ]:
iris.head(5)

In [ ]:
x = iris.sort_values(by = "distanza", ascending=True).head(1)

In [ ]:
print(x)

In [ ]:
#oppure
minima_distanza = iris["distanza"].min()
x = iris.query("distanza == @minima_distanza")
print(x)

## Usiamo una nostra libreria

In [ ]:
import libreria_prevedi_iris as lp

In [ ]:
nuovo_iris = [5.9,3.0,5.1,1.8]

In [ ]:
lp.prevedi_iris(nuovo_iris)

## Usiamo scikit-learn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
knc = KNeighborsClassifier(n_neighbors=1)

In [ ]:
#per eliminare il warning usare target_training.values.ravel()
knc.fit(X = iris[["sepal_length","sepal_width","petal_length","petal_width"]],
        y = iris[["species"]])

In [ ]:
nuovo_iris = pd.DataFrame(data = [nuovo_iris],
                          columns = ["sepal_length","sepal_width","petal_length","petal_width"] )

In [ ]:
nuovo_iris

In [ ]:
knc.predict(X = pd.DataFrame(nuovo_iris))

In [ ]:
knc.kneighbors(nuovo_iris)

## LLM con Open AI

In [ ]:
import pandas as pd
from openai import OpenAI
import os
import getpass

In [ ]:
secret_key = getpass.getpass(prompt='Inserisci la chiave: ')

In [ ]:
os.environ['OPENAI_API_KEY']=secret_key

Inizializziamo le api

In [ ]:
client = OpenAI()

Proviamo a fare una chiamata

In [ ]:
completition = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "Sei un'assistente AI"},
    {"role": "user", "content": "Come puoi aiutarmi con pandas?"}
  ]
)

In [ ]:
print(completition.choices[0].message.content)

## Applicazione pratica con LLM

Acquisiamo il dataframe fatture

In [ ]:
fatture = pd.read_csv(filepath_or_buffer = r"C:\Users\ianto\Desktop\Corso python\file\Fatture.csv",
        sep = ";",  #separtore del file
        header = None,
        names = ["NumeroFattura","Tipologia","Importo","Iva","IdCliente","Regione","DataFattura","NumeroFornitore"],
        decimal = ",",
        )

fatture['DataFattura'] =  pd.to_datetime(fatture['DataFattura'],
                                         format="%d/%m/%Y"
                                        )  

fatture['NumeroFornitore'] = fatture['NumeroFornitore'].astype('Int64')

Proviamo a chiedere all'LLM di generare un codice pandas

In [ ]:
risposta = client.chat.completions.create(
  model="gpt-4o",
  messages=[
            {"role": "system",
             "content": "Supporti l'utente scrivendo solo codice Python eseguibile con la libreria pandas per rispondere alla sua richiesta" },
            {"role": "user",
             "content": f"""
            Stai lavorando con un dataframe Pandas in Python.
            Il nome del dataframe è df.
            
            Questo è l'elenco delle colonne di df:
            {str(fatture.columns.values)}

            Considera questa richiesta: qual è la fattura più recente?
            
            Segui queste istruzioni:
            1. Converti la richiesta precedente in codice Python eseguibile usando Pandas.
            2. L'ultima riga di codice dovrebbe essere un'espressione Python che può essere chiamata con la funzione `eval()`.
            3. Il codice dovrebbe rappresentare una soluzione alla query.
            4. Restituisci come risposta soltanto il codice Python, non aggiungere altre considerazioni o preamboli
            5. La risposta non deve iniziare con ```python
            6. Non creare dataframe d'esempio
            7. Non fare nessun commento
            8. Rispondi esclusivamente con codice Python
            9. Non usare eval nella risposta
            10. Non creare esempi di dati, usa direttamente nella risposta il nome del dataframe è df 
             """}],
            temperature=0.2)

In [ ]:
print(risposta.choices[0].message.content)

Creiamo una funzione che richiede in input un dataframe e un'interrogazione in linguaggio naturale

In [ ]:
def generete_pandas_code(df, query):
    user_content = f"""
            Stai lavorando con un dataframe Pandas in Python.
            Il nome del dataframe è df.
            
            Questo è l'elenco delle colonne di df:
            {str(df.columns.values)}

            Considera questa richiesta: {query}?
            
            Segui queste istruzioni:
            1. Converti la richiesta precedente in codice Python eseguibile usando Pandas.
            2. L'ultima riga di codice dovrebbe essere un'espressione Python che può essere chiamata con la funzione `eval()`.
            3. Il codice dovrebbe rappresentare una soluzione alla query.
            4. Restituisci come risposta soltanto il codice Python, non aggiungere altre considerazioni o preamboli
            5. La risposta non deve iniziare con ```python
            6. Non creare dataframe d'esempio
            7. Non fare nessun commento
            8. Rispondi esclusivamente con codice Python
            9. Non usare eval nella risposta
            10. Non creare esempi di dati, usa direttamente nella risposta il nome del dataframe è df 
            """
    domanda = client.chat.completions.create(
          model="gpt-4o",
          messages=[
                    {"role": "system",
                    "content": "Supporti l'utente scrivendo solo codice Python eseguibile con la libreria pandas per rispondere alla sua richiesta" },
                    {"role": "user",
                     "content":user_content }
                   ])

    risposta = domanda.choices[0].message.content
    return risposta

In [ ]:
df_input = fatture

In [ ]:
query_input = input("Chiedi la generazione di un codice Pandas. Scrivi x per uscire")

while query_input!="x":
    risposta = generete_pandas_code(df=df_input, query=query_input)
    print(risposta)
    query_input = input("Chiedi la generazione di un codice Pandas. O scrivi x per uscire")

# LLM in locale

In [ ]:
import ollama

Elenco modelli installati in locale

In [ ]:
ollama.list()

Chiamata in locale, può essere molto lenta

In [ ]:
response =  ollama.chat(model="gemma3:1b",
                         messages=[
                                    {"role": "system", "content": "Sei un'assistente AI"},
                                    {"role": "user", "content": "Qual è la capitale dell'italia"}
                                  ]
                       )

In [ ]:
print(response["message"]["content"])

# Python e Database

In [ ]:
import getpass
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
import json
with open(r"C:\Users\ianto\Desktop\Corso Python\config.json", 'r') as config_file:
    config_data = json.load(config_file)

server_name = config_data['server_name']
database_name = config_data['database_name']
utente = config_data['utente']
password = config_data['password']

#utente = getpass.getpass(prompt='Inserisci nome utente: ')
#password = getpass.getpass(prompt='Inserisci la password: ')

In [ ]:
conn_str = f'mssql+pyodbc://{utente}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&encrypt=yes'

In [ ]:
engine = create_engine(conn_str)

In [ ]:
df = pd.read_sql( sql = "SELECT TOP 10 * FROM Clienti WHERE Regione = 'Lombardia'", 
             con = engine,
             parse_dates = "DataNascita")
df

Scriviamo ora da Python al Database!

In [ ]:
df.to_sql(name = "ClientiPython", 
          con = engine, 
          if_exists="append",
          index=False)

Chiudiamo la connessione

In [ ]:
engine.dispose()

Attenzione, per stabilire una connessione crittografata occorre un certificato sul DB. Spesso le stringhe di connessione non hanno questa opzione abilitata di default

In [ ]:
import json
with open(r"C:\Users\ianto\Desktop\Corso Python\config_2.json", 'r') as config_file:
    config_data = json.load(config_file)

server_name = config_data['server_name']
database_name = config_data['database_name']
utente = config_data['utente']
password = config_data['password']

#utente = getpass.getpass(prompt='Inserisci nome utente: ')
#password = getpass.getpass(prompt='Inserisci la password: ')

In [ ]:
conn_str = f'mssql+pyodbc://{utente}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&encrypt=yes'

In [ ]:
engine = create_engine(conn_str)

La prossima istruzione restituisce l'errore


Restituisce errore
OperationalError: (pyodbc.OperationalError) ('08001', "[08001] [Microsoft][ODBC Driver 17 for SQL Server]Provider SSL: Catena di certificati emessa da una Autorità di certificazione non disponibile nell'elenco locale.\r\n (-2146893019) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Il client non è in grado di effettuare la connessione. (-2146893019)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
df = pd.read_sql( sql = "SELECT TOP 10 * FROM Clienti WHERE Regione = 'Lombardia'", 
             con = engine,
             parse_dates = "DataNascita")
df

In [ ]:
engine.dispose()